# Requirment 

In [ ]:
!python.exe -m pip install --upgrade pip

In [216]:
!pip install pytesseract
!pip install beautifulsoup4
!pip install pandas
# !pip install tensorflow
!pip install Pillow
!pip install lxml
!pip install opencv-python
!pip install pathlib
!pip install ast

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\User\AppData\Local\Temp\pip-install-cmvjdell\ast_af27a5a016f54dbf817e7d342a0d55fb\setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
        File "C:\Users\User\AppData\Local\Programs\Python\Python38\lib\codecs.py", line 905, in open
          file = builtins.open(filename, mode, buffering)
      FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-cmvjdell\\ast_af27a5a016f54dbf817e7d342a0d55fb\\AST/README'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encount

# Import

In [1]:
from bs4 import BeautifulSoup
import base64
import pandas as pd
# import tensorflow as tf
from pytesseract import pytesseract
from PIL import Image
import glob
import cv2
import numpy as np
from pathlib import Path

# Main

In [73]:
%cd C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\

for file in glob.iglob('data\department\*交叉查榜*.html'):
    # print(file)
    processing_department(file)

c:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students
長庚大學中醫學系
      rank exam_location     stuno      univ year  \
0     正1\n        私立復旦高中  111****5  長庚大學中醫學系  112   
1     正三\n      臺北市立華江高中  110****2  長庚大學中醫學系  112   
2    正……\n      高雄市立左營高中  113****5  長庚大學中醫學系  112   
3     正7\n        國立虎尾高中  112****2  長庚大學中醫學系  112   
4     正9\n        國立臺南一中  112****2  長庚大學中醫學系  112   
5    正11\n      臺北市立百齡高中  110****9  長庚大學中醫學系  112   
6    正13\n      臺北市立華江高中  110****7  長庚大學中醫學系  112   
7   正1……\n      國立臺灣師大附中  110****8  長庚大學中醫學系  112   
8    正17\n        國立斗六高中  112****7  長庚大學中醫學系  112   
9    正19\n      桃園市立武陵高中  111****4  長庚大學中醫學系  112   
10   正21\n      新北市立錦和高中  111****2  長庚大學中醫學系  112   
11   正23\n      臺北市立中山女中  110****6  長庚大學中醫學系  112   
12  正2……\n      國立臺灣師大附中  110****1  長庚大學中醫學系  112   
13   正27\n      高雄市立左營高中  113****5  長庚大學中醫學系  112   
14   正29\n        國立嘉義女中  112****4  長庚大學中醫學系  112   
15    正1\n        國立馬祖高中  113****3  長庚大學中醫學系  112   
16

KeyboardInterrupt: 

## Functions

In [72]:
def processing_department(filename):
    with open(filename, 'r', encoding="utf-8") as file:
        html = file.read()
        # print(html)
        soup = BeautifulSoup(html, 'html.parser')
    
    fields = ['rank', 'exam_location', 'stuno', 'univ',  'year', 'department_name', 'select']
    df = pd.DataFrame(columns=fields)
    df1 = pd.DataFrame(columns=fields)
    df2 = pd.DataFrame(columns=fields)
    
    department_name = []
    select = []

    str = soup.head.find_all('title')[0].text
    pos = str.find('年')
    year = str[pos-3:pos]  # str[5:8] str[5], str[6], str[7]

    str = soup.head.find_all('title')[0].text
    pos = str.find('-')
    univ = str[:pos].rstrip()

    dark_list = soup.body.find_all(bgcolor="#DEDEDC") # 褐色
    df1 = processing_list(univ, year, dark_list, df, department_name, select)
    print(df1)
    
    white_list = soup.body.find_all(bgcolor="#FFFFFF") # 白色
    df2 = processing_list(univ, year, white_list, df, department_name, select)
    
    # make two df cross-interleave
    df = cross_interleave(df1, df2)

    # explode list into df
    df = df.explode(['department_name', 'select']).reset_index(drop=True)
    df = df.dropna()
    
    # save to csv
    result_dir = Path(r'.\result')
    df.to_csv(path_or_buf = result_dir / f'{univ}.csv', encoding='utf-8-sig', index=False)
    

### Functions

In [71]:
def processing_list(univ, year, color_list, df, department_name, select):
    

    rank = ''
    stuno = ''
    
    
    print(univ)
    for item in color_list:
        color_list_item_scope1 = item.children # get message under the soup.body.find_all(bgcolor="")
        count_item = 0
        exam_location = ""
        for item_scope2 in color_list_item_scope1:
            # print("i:", item_scope2," ", count_item)
            """
            count_item:3 = 正備取
            count_item:5 = number
            count_item:9 = 錄取校系
            """
            if count_item == 3:
                rank = img_detect(item_scope2 = item_scope2, img = r'data/rank.png', langage='chi_sim', oem_psm = '--oem 0 --psm 7')

            elif count_item == 5:
                stuno = img_detect_(item_scope2)
                # stuno = img_detect(item_scope2 = item_scope2, img = r'data/num.png', langage='eng', oem_psm = '--oem 1 --psm 7')
                
                # get exam location
                x = item.find_all('a')
                exam_location = x[0].text[6:-1]
                
            elif count_item == 9:
                tr_scope3 = item_scope2.find_all('tr')
                department_name = []
                select = []
                for item_scope4 in tr_scope3:
                    # print("item_scope4:", count, rank, item_scope4)
                    count_item_scope5 = 0
                    for item_scope5 in item_scope4:
                        # print('item_scope5:', count_, item_scope5)
                        """_summary_
                        count_item_scope5: 1 徽章
                        count_item_scope5: 3 大學
                        count_item_scope5: 5 校系排名
                        """
                        if count_item_scope5 == 1:
                            img_tag = item_scope5.find('img')
                            # print(img_tag)
                            if img_tag is not None:
                                img_tag_ = True
                            else:
                                img_tag_ = False
                            select.append(img_tag_)
                            
                        elif count_item_scope5 == 3:
                            # We'll drop later to suit up with img_tag_
                            if item_scope5.text != "":
                                department_name.append(item_scope5.text)
                            else:
                                department_name.append(np.nan)
                            
                        elif count_item_scope5 == 5:
                            # to get the rank of each dept
                            pass
                        
                        count_item_scope5 += 1

            count_item += 1
        
        dict = {'rank': rank, 'exam_location': exam_location, 'stuno': stuno, 'univ': univ, 'year': year, 'department_name': department_name, 'select': select}
        df = pd.concat([df, pd.DataFrame([dict])], ignore_index = True)
    
    return df

def cross_interleave(df1, df2):
    # make two df cross-interleave
    df = pd.DataFrame()
    max_len = max(len(df1), len(df2))
    for line in range(max_len):
        if line < len(df1):
            df = pd.concat([df, df1.iloc[[line]]], ignore_index=True)
        if line < len(df2):
            df = pd.concat([df, df2.iloc[[line]]], ignore_index=True)
    return df

#### Function

In [70]:
def img_detect(item_scope2, img, langage, oem_psm):
    try:
        base64_str = item_scope2.find_all('img')[0]['src'].split(',')[1]
    except IndexError:
        return "None"
    else:
        with open(img, "wb") as fh:
            fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
        path_to_tesseract = Path(r'C:\Program Files\Tesseract-OCR\tesseract.exe')
        pytesseract.tesseract_cmd = path_to_tesseract
        path_to_image = img
        cv2.imwrite(img, thresholding_(path_to_image))
        return pytesseract.image_to_string(Image.open(path_to_image), lang=langage, config=oem_psm)
    
def img_detect_(item_scope2):
    # start from (4, 20)
    basic_width = 4
    # 9 pixels each
    digit_width = 9
    result = ''
    try:
        base64_str = item_scope2.find_all('img')[0]['src'].split(',')[1]
    except IndexError:
        return "None"
    else:
        with open(r'data/num.png', "wb") as fh:
            fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
        img_ = cv2.imread(r'data/num.png')
        for i in range(1,9):
            crop_initial = digit_width*(i-1)+basic_width
            crop_end = digit_width*i+basic_width
            crop_img = img_[0: 20, crop_initial:crop_end]
            cv2.imwrite(f'data/crop_img{i}.png', crop_img)
            image = f'data/crop_img{i}.png'
            path_to_image = image
            cv2.imwrite(f'data/crop_img{i}.png', thresholding_(image))
            path_to_tesseract = Path(r'C:\Program Files\Tesseract-OCR\tesseract.exe')
            pytesseract.tesseract_cmd = path_to_tesseract    
            result = str(result + pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')).strip()
    return result


##### Function

In [30]:
def thresholding_(png):
    image = cv2.imread(png)
    gray = get_grayscale(image)
    thresh_ = thresholding(gray)
    return thresh_
  

###### Function

In [31]:
# Convert the image to grayscale
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Remove noise from the image
def remove_noise(image):
    return cv2.medianBlur(image, 5)
 
# Thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Dilation
def dilate(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.dilate(image, kernel, iterations=1)
    
# Erosion
def erode(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.erode(image, kernel, iterations=1)

# Opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# Canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

# Skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

# Number

In [21]:
# {exam_location, stuno}
scho = '國立'
# scho = exam_location
# for file in glob.iglob(f'data\test_location\{scho}學測查榜.html'):
# stuno_first3 = stuno[0:3]
# stuno_last = stuno[-1]

# get exam_lacation stuno & list of univs, merge it into a list -> [stuno[0:3], stuno[-1], dept....]
# make the same of the original version
# if list_exam_location[i] == list_original[i]:
# get the whole number and return
for file in glob.iglob(f'data\test_location\{scho}*學測查榜*.html'):
    processing_numbers(file)
    
    

In [35]:
def processing_numbers(filename):
    with open(filename, 'r', encoding="utf-8") as file:
        html = file.read()
        # print(html)
        soup = BeautifulSoup(html, 'html.parser')
        
    fields = ['rank', 'exam_location', 'stuno', 'univ',  'year', 'department_name', 'select']
    df = pd.DataFrame(columns=fields)

### data_format

In [65]:
str1 = 'lL1x%\"«oO'

char_to_replace = {'l':'1', 'L':'1', 'x':'*', '%':'*', '\"':'*', '«':'*', 'o':'0', 'O':'0'}

# Iterate over all key-value pairs in dictionary 
for key, value in char_to_replace.items():
    # Replace key character with value character in string
    str1 = str1.replace(key, value)     

print(str1)
print(type(str1))

111****00
<class 'str'>


### test img_cropping

In [17]:
from pytesseract import pytesseract
from PIL import Image
import cv2
from pathlib import Path

# target
img = cv2.imread(r'C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\data\num.png')

print(img.shape)

# 從(4, 20)開始
basic_width = 4
# 每個間格9 pixel
digit_width = 9

rlt = ''

def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


# 8 digits
for i in range(1, 9):
    crop_initial = digit_width*(i-1)+basic_width
    crop_end = digit_width*i+basic_width
    crop_img = img[0: 20, crop_initial:crop_end]
    print(crop_img.shape)
    print(type(i), i)
    cv2.imwrite(f'data/crop_img{i}.png', crop_img)
    # cv2.imshow("img", crop_img)
    # cv2.waitKey(0)
    image = f'data/crop_img{i}.png'
    path_to_image = image
    image_ = cv2.imread(image)
    gray = get_grayscale(image_)
    thresh_ = thresholding(gray)
    cv2.imwrite(f'data/crop_img{i}.png', thresh_)
    path_to_tesseract = Path(r'C:\Program Files\Tesseract-OCR\tesseract.exe')
    pytesseract.tesseract_cmd = path_to_tesseract    
    rlt = str(rlt + pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')).strip()

print(rlt)


(20, 90, 3)
(20, 9, 3)
<class 'int'> 1
(20, 9, 3)
<class 'int'> 2
(20, 9, 3)
<class 'int'> 3
(20, 9, 3)
<class 'int'> 4
(20, 9, 3)
<class 'int'> 5
(20, 9, 3)
<class 'int'> 6
(20, 9, 3)
<class 'int'> 7
(20, 9, 3)
<class 'int'> 8
111****4
